# FifaSkill

A Probabalistic Programming Package for European Soccer Analysis by Vinay Ramesh (vrr2112) and Alek Anchowski (aja2173)


In [6]:
import fifaskill
import numpy as np
import pandas as pd
import sqlite3
import tensorflow as tf

from edward.models import Normal, Poisson

## Introduction

Trueskill, developed by Microsoft Research, is a general skill based ranking system based upon a prior distribution around a player's mean skill and variance (Herbrich, Minka, Graepel). The original system models an player's skill, variance in performance, and uses an Expectation Propogation algorithm to update a player's skill representation and variance for every new game that the player participates in.

In our project fifaskill, we extend the Trueskill method to FIFA games, with the aim of modeling a team's overall skill and predicting future matches. We begin with the basic true skill model, and experiment with modeling offense and defense skills seperately, as well as home team advantage. 

## Data

We use the [Kaggle Dataset for European Soccer Matches](https://www.kaggle.com/hugomathien/soccer) 
which consists of a list of matches in x leagues and x seasons. We are interested in predicting outcomes of future matches using this given data.

In [12]:
db = '../database.sqlite'
query_fname = '../db_queries/detailed_match_query.sql'

conn = sqlite3.connect(db)

q_file = open(query_fname, 'r')
data = pd.read_sql(q_file.read(), conn)

data.head(5)

,id,country_name,league_name,season,stage,date,home_team,away_team,home_team_goal,away_team_goal
0,1730,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,Arsenal,West Bromwich Albion,1,0
1,1731,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,Sunderland,Liverpool,0,1
2,1732,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,West Ham United,Wigan Athletic,2,1
3,1734,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,Everton,Blackburn Rovers,2,3
4,1735,England,England Premier League,2008/2009,1,2008-08-16 00:00:00,Middlesbrough,Tottenham Hotspur,2,1


## Model

### Basic

To model a team's skill, we assume a Gaussian prior, where the probability of skill $s$ is 
$$
\begin{align*}
  p(\mathbf{s})
  &=
  \text{Normal}( 25, \frac{25}{3} ^ 2)
\end{align*}
$$

We then model the performance of a team based on its skill, with another parameter $\beta$ describing the variance of a particular team's performance.

$$
\begin{align*}
  p(\mathbf{s})
  &=
  \text{Normal}( s, \beta)
\end{align*}
$$

Given two team's performances, $s1$ and $s2$, we then model the game's outcome, r, which we define to be the difference of the goals scored by each team against one another. In the model, this corrosponds to drawing from a Poisson distribution where the rate is the performance of the team. 

To train the model, given $n$ teams, we obtain a matrix $R^{n, n}$ from the database, where an entry $R[team1, team2]$ would corrospond to the average difference of the goals scored by team1 against team2 and goals scored by team2 against team1. R is skew-symmetric, meaning $R^T = -R$, or more specifically that $R[team1, team2] == -R[team2, team1]$.

Now let's write the model in Edward.

In [19]:
n = len(data.home_team.unique())

initial_loc = tf.zeros((n, 1), dtype='float32') 
initial_scale = tf.ones((n, 1),  dtype='float32')

team_skill = Normal(loc=initial_loc, scale=initial_scale)

team_performance = Normal(loc=team_skill, scale=initial_scale)

perf_diff_tmp = tf.tile(tf.reduce_sum(team_performance, 1,
                    keepdims=True),
                    [1, n])
perf_diff = perf_diff_tmp - tf.transpose(perf_diff_tmp)

### Offense Defense Separation

To extend the basic model we seperate the team's skill into two scores, offense and defense, where each is also defined with a Gaussian prior. Each team also has a variable offense and defense performance, based of the corrosponding skill, similar to how skill and performance were related in the basic model. Based on the offense and defense performance, we again draw from a Poisson distribution twice to determine the number of goals scored and number of goals allowed.

Next we define the new model in Edward.

In [ ]:
team_off = Normal(loc=initial_loc, scale=initial_scale)
team_def = Normal(loc=initial_loc, scale=initial_scale)

team_off_performance = Normal(loc=team_off, scale=initial_scale)
team_def_performance = Normal(loc=team_off, scale=initial_scale)

off_tile = tf.tile(tf.reduce_sum(
        team_off_performance, 1, keepdims=True), [1, n])
def_tile = tf.tile(tf.reduce_sum(
        team_def_performance, 1, keepdims=True), [1, n])
goals_scored_performance = off_tile - tf.transpose(def_tile)
goals_allowed_performance = def_tile - tf.transpose(off_tile)

goals_scored = Poisson(rate=goals_scored_performance)
goals_allowed = Poisson(rate=goals_allowed_performance)

perf_diff = goals_scored - goals_allowed

## Inference

Perform variational inference.
Define the variational model to be a fully factorized normal.

In [5]:
qf = Normal(loc=tf.Variable(tf.random_normal([N])),
            scale=tf.nn.softplus(tf.Variable(tf.random_normal([N]))))

Run variational inference for `500` iterations.

In [6]:
inference = ed.KLqp({f: qf}, data={X: X_train, y: y_train})
inference.run(n_iter=5000)

5000/5000 [100%] ██████████████████████████████ Elapsed: 9s | Loss: 82.755


In this case
`KLqp` defaults to minimizing the
$\text{KL}(q\|p)$ divergence measure using the reparameterization
gradient.
For more details on inference, see the [$\text{KL}(q\|p)$ tutorial](/tutorials/klqp).
(This example happens to be slow because evaluating and inverting full
covariances in Gaussian processes happens to be slow.)

## Model Evaluation

To evaluate our model's prediction for the offense and defense scores for each team, we test them against the true values as defined by our dataset. For offense, we take an average of the chance creation scores of a team across several seasons, and for defense we average the defense pressure and aggression scores. For future work, it might be useful to determine whether our choice to combine these particular scores exactly corrosponds to a pure offense/defense skill level - perhaps there is a more complex relationship between pressure and aggression for example that would corrospond to higher defense scores.

In [13]:
query_fname = '../db_queries/off_def.sql'

q_file = open(query_fname, 'r')

off_def = pd.read_sql(q_file.read(), conn)
off_def['offense'] = off_def[['pass', 'cross', 'shoot']].mean(axis=1)
off_def['defense'] = off_def[['pressure', 'aggression']].mean(axis=1)
off_def = off_def[['team_name', 'offense','defense']]

In [14]:
off_def.head(5)

,team_name,offense,defense
0,Arsenal,42.500000,48.500000
1,Aston Villa,52.722222,42.750000
2,Birmingham City,58.777778,48.750000
3,Blackburn Rovers,51.666667,50.333333
4,Blackpool,58.833333,49.250000


Next, we compare these scores to our predicted values

## Criticism/ Model Revision